#### By: Peyman Shahidi
#### Created: Oct 29, 2025
#### Last Edit: Nov 10, 2025

<br>

In [104]:
#Python
import getpass
import numpy as np
import pandas as pd
from collections import defaultdict
import itertools
import random 

## formatting number to appear comma separated and with two digits after decimal: e.g, 1000 shown as 1,000.00
pd.set_option('float_format', "{:,.2f}".format)

import matplotlib.pyplot as plt
#%matplotlib inline
#from matplotlib.legend import Legend

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 200)

In [105]:
#Python
import getpass
import numpy as np
import pandas as pd
from collections import defaultdict
import itertools
import random 

## formatting number to appear comma separated and with two digits after decimal: e.g, 1000 shown as 1,000.00
pd.set_option('float_format', "{:,.2f}".format)

import matplotlib.pyplot as plt
#%matplotlib inline
#from matplotlib.legend import Legend

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 200)

main_folder_path = ".."
input_data_path = f"{main_folder_path}/data"
# Modify the output path accordingly
output_data_path = f'{input_data_path}/computed_objects/BLS_ONET_matchedEmpShares'
output_plot_path = f"{main_folder_path}/writeup/plots/anthropic_AI_index/BLS_ONET_matchedEmpShares"

# Toggle: if True, randomly reassign occ_totalEmpShare weights in the merged master_df
# during the merge_industry_employment_shares step. Set to False for default behavior.
randomize_occ_weights = True


In [106]:
import os
for path in [output_data_path, output_plot_path]:
    if not os.path.exists(path):
        os.makedirs(path)

In [107]:
my_sector = '4-digit' 
my_onet_level = 'detailed'
onet_occupation_code_var = 'Detailed_Occupation_Code'
onet_occupation_title_var = 'Detailed_Occupation_Title'

#### Main Code

In [108]:
# Read the merged data
merged_data = pd.read_csv(f"{input_data_path}/computed_objects/ONET_Eloundou_Anthropic_GPT/ONET_Eloundou_Anthropic_GPT.csv")

In [109]:
# # Drop the supplemental tasks
# merged_data = merged_data[merged_data['Task Type'] != 'Supplemental'].reset_index(drop=True)

# # Drop rows whose Occupation Title includes 'Teachers, Postsecondary'
# merged_data = merged_data[~merged_data['Occupation Title'].str.contains('Teachers, Postsecondary')].reset_index(drop=True)

In [110]:
handoff_var = 'task_semantic_dispersion'
ai_outcome_var = 'ai_fraction'
ai_exposure_var = 'human_E1_fraction'
fragmentation_index_var = 'fragmentation_index'

# Create occupation-level data
occupation_analysis = pd.read_csv(f"{input_data_path}/computed_objects/fragmentationIndex/occupation_analysis_with_fragmentationIndex.csv")

# Merge with embedding-based handoff measure
occupation_task_dispersion = pd.read_csv(f"{input_data_path}/computed_objects/ONET_Eloundou_Anthropic_GPT/ONET_occupation_task_semantic_dispersion.csv")

occupation_analysis = occupation_analysis.merge(occupation_task_dispersion[[ 'O*NET-SOC Code', 'Occupation Title', 'task_semantic_dispersion']],
                                                on=['O*NET-SOC Code', 'Occupation Title'], how='left')


# Read OG occupation analysis with SOC mappings
ONET = pd.read_csv(f"{input_data_path}/computed_objects/ONET_cleaned_tasks.csv")

# Keep only the relevant 
SOC_mappings = ONET[['O*NET-SOC Code', 'Occupation Title',
                     'Major_Group_Code', 'Major_Group_Title',
                     'Minor_Group_Code', 'Minor_Group_Title',
                     'Broad_Occupation_Code', 'Broad_Occupation_Title',
                     'Detailed_Occupation_Code', 'Detailed_Occupation_Title']].copy()
SOC_mappings = SOC_mappings.drop_duplicates(subset=['O*NET-SOC Code', onet_occupation_code_var])

# Merge SOC levels with the occupation analysis
occupation_analysis = occupation_analysis.merge(SOC_mappings, on=['O*NET-SOC Code', 'Occupation Title'], how='left')

# Aggregate to onet_occupation_code_var level
occupation_analysis = occupation_analysis.groupby([onet_occupation_code_var, onet_occupation_title_var]).agg({
    handoff_var: 'mean',
    ai_outcome_var: 'mean',
    ai_exposure_var: 'mean',
    fragmentation_index_var: 'mean',
    'num_tasks': 'sum'
}).reset_index()

# Drop occupations with null handoff measure
print(f"Before dropping null handoff measure: {occupation_analysis.shape[0]} occupations")
occupation_analysis = occupation_analysis[~occupation_analysis[handoff_var].isnull()]
print(f"After dropping null handoff measure: {occupation_analysis.shape[0]} occupations")
occupation_analysis.sort_values(by=handoff_var, ascending=False)

Before dropping null handoff measure: 758 occupations
After dropping null handoff measure: 758 occupations


,Detailed_Occupation_Code,Detailed_Occupation_Title,task_semantic_dispersion,ai_fraction,human_E1_fraction,fragmentation_index,num_tasks
311,29-1292,Dental Hygienists,0.69,0.06,0.00,1.00,18
439,41-9012,Models,0.68,0.00,0.07,1.00,14
488,43-9031,Desktop Publishers,0.68,0.44,0.06,0.78,18
703,51-9141,Semiconductor Processing Technicians,0.67,0.00,0.04,1.00,24
498,45-2041,"Graders and Sorters, Agricultural Products",0.66,0.00,0.00,1.00,6
...,...,...,...,...,...,...,...
612,49-9098,"Helpers--Installation, Maintenance, and Repair...",0.42,0.00,0.00,1.00,16
13,11-3131,Training and Development Managers,0.42,0.33,0.08,1.00,12
28,11-9161,Emergency Management Directors,0.41,0.04,0.39,1.00,23
256,27-2031,Dancers,0.40,0.00,0.00,1.00,14


## Add BLS Employment Share Weights

In [111]:
# Add BLS employment shares for all NAICS sectors and create a master dataset
bls_sector_shares = pd.read_csv(f'{input_data_path}/computed_objects/BLS_ONET_empShares/BLS{my_sector}_ONET{my_onet_level}_empShares.csv')

master_df = occupation_analysis.copy()


# 1) sectorEmpShare
sector_weights_df = bls_sector_shares[['NAICS', 'NAICS_TITLE', 'TOT_EMP']].groupby(['NAICS', 'NAICS_TITLE']).sum('TOT_EMP')

# Convert to % and change variable name
sector_weights_df['sectorEmpShare'] = sector_weights_df['TOT_EMP'] / sector_weights_df['TOT_EMP'].sum()

# Merge back sector weights to bls dataset to get sector-by-sector occupation data with sector weights
bls_sector_weights_df = bls_sector_shares[['NAICS', 'NAICS_TITLE', 'OCC_CODE', 'OCC_TITLE']].merge(sector_weights_df, on='NAICS', how='left')

# Add weight column to master_df
master_df = master_df.merge(bls_sector_weights_df[['NAICS', 'NAICS_TITLE', 'OCC_CODE', 'sectorEmpShare']], left_on=onet_occupation_code_var, right_on=['OCC_CODE'], how='left')
master_df = master_df.drop(columns=['OCC_CODE'])


# 2) occ_sectorEmpShare
within_sector_weights_df = bls_sector_shares[['NAICS', 'NAICS_TITLE', 'OCC_CODE', 'OCC_TITLE', 'TOT_EMP']].copy()
within_sector_weights_df['occ_sectorEmpShare'] = within_sector_weights_df['TOT_EMP'] / within_sector_weights_df.groupby(['NAICS', 'NAICS_TITLE'])['TOT_EMP'].transform('sum')

# Add weight column to master_df
master_df = master_df.merge(within_sector_weights_df[['NAICS', 'OCC_CODE', 'occ_sectorEmpShare']], left_on=['NAICS', onet_occupation_code_var], right_on=['NAICS', 'OCC_CODE'], how='left')
master_df = master_df.drop(columns=['OCC_CODE'])

# Drop rows with null NAICS -- i.e., unmatched occupations in both procedures
print(f"Dropping {master_df.NAICS.isnull().sum()} rows with null NAICS (unmatched occupations)")
master_df = master_df[~master_df.NAICS.isnull()]


# Save master dataframe to CSV
master_df = master_df.sort_values(by=['NAICS', 'occ_sectorEmpShare'], ascending=[True, False]).reset_index(drop=True)
out_dir = f"{output_data_path}"#/BLS{my_sector}_ONET{my_onet_level}/{dependent_var}/"
os.makedirs(out_dir, exist_ok=True)
master_out = f"{out_dir}/BLS{my_sector}_ONET{my_onet_level}.csv"
master_df.to_csv(master_out, index=False)
display(master_df.head())

Dropping 20 rows with null NAICS (unmatched occupations)


,Detailed_Occupation_Code,Detailed_Occupation_Title,task_semantic_dispersion,ai_fraction,human_E1_fraction,fragmentation_index,num_tasks,NAICS,NAICS_TITLE,sectorEmpShare,occ_sectorEmpShare
0,45-4022,Logging Equipment Operators,0.64,0.00,0.00,1.00,9,113300,Logging,0.00,0.42
1,53-3032,Heavy and Tractor-Trailer Truck Drivers,0.58,0.03,0.03,1.00,31,113300,Logging,0.00,0.18
2,45-4021,Fallers,0.54,0.00,0.11,1.00,19,113300,Logging,0.00,0.07
3,45-1011,"First-Line Supervisors of Farming, Fishing, an...",0.55,0.00,0.00,1.00,27,113300,Logging,0.00,0.06
4,43-9061,"Office Clerks, General",0.52,0.38,0.24,0.86,21,113300,Logging,0.00,0.04


## Define Weighted Hand-off Measure

In [112]:
# 1) Ensure numeric types and fill NaN with 0 (safe for summing)
# master_df['num_tasks'] = pd.to_numeric(master_df.get('num_tasks', 0), errors='coerce').fillna(0)
master_df['occ_sectorEmpShare'] = pd.to_numeric(master_df.get('occ_sectorEmpShare', 0), errors='coerce').fillna(0)

# 2) Per-row weighted value
master_df[f'{handoff_var}_weighted'] = master_df[handoff_var] * master_df['occ_sectorEmpShare']
master_df[f'{ai_outcome_var}_weighted'] = master_df[ai_outcome_var] * master_df['occ_sectorEmpShare']
master_df[f'{ai_exposure_var}_weighted'] = master_df[ai_exposure_var] * master_df['occ_sectorEmpShare']
master_df[f'{fragmentation_index_var}_weighted'] = master_df[fragmentation_index_var] * master_df['occ_sectorEmpShare']
master_df['num_tasks_weighted'] = master_df['num_tasks'] * master_df['occ_sectorEmpShare']

# 3) Compute group sums within NAICS / NAICS_TITLE for both columns
named_aggs = {
    'occ_sectorEmpShare_sum': ('occ_sectorEmpShare', 'sum')
    }
named_aggs[f'{handoff_var}_weighted_sum'] = (f'{handoff_var}_weighted',  'sum')
named_aggs[f'{ai_outcome_var}_weighted_sum'] = (f'{ai_outcome_var}_weighted', 'sum')
named_aggs[f'{ai_exposure_var}_weighted_sum'] = (f'{ai_exposure_var}_weighted', 'sum')
named_aggs[f'{fragmentation_index_var}_weighted_sum'] = (f'{fragmentation_index_var}_weighted', 'sum')
named_aggs['num_tasks_weighted_sum'] = ('num_tasks_weighted', 'sum')


sector_sums = (
    master_df.groupby(['NAICS','NAICS_TITLE'], dropna=True)
             .agg(**named_aggs)
             .reset_index()
)

# Calculate Hand-off measure: {variable}_weighted_sum / occ_sectorEmpShare_sum (Cause occ_sectorEmpShare_sum might be < 1)
sector_sums[f'{handoff_var}_weighted'] = sector_sums[f'{handoff_var}_weighted_sum'] / sector_sums['occ_sectorEmpShare_sum']
sector_sums[f'{ai_outcome_var}_weighted'] = sector_sums[f'{ai_outcome_var}_weighted_sum'] / sector_sums['occ_sectorEmpShare_sum']
sector_sums[f'{ai_exposure_var}_weighted'] = sector_sums[f'{ai_exposure_var}_weighted_sum'] / sector_sums['occ_sectorEmpShare_sum']
sector_sums[f'{fragmentation_index_var}_weighted'] = sector_sums[f'{fragmentation_index_var}_weighted_sum'] / sector_sums['occ_sectorEmpShare_sum']
sector_sums['num_tasks_weighted'] = sector_sums['num_tasks_weighted_sum'] / sector_sums['occ_sectorEmpShare_sum']

# Drop intermediate sum columns
sector_sums = sector_sums.drop(columns=['occ_sectorEmpShare_sum', 
                                        f'{handoff_var}_weighted_sum', 
                                        f'{ai_outcome_var}_weighted_sum', 
                                        f'{ai_exposure_var}_weighted_sum',
                                        f'{fragmentation_index_var}_weighted_sum',
                                        'num_tasks_weighted_sum'])


# # Optional: sort sectors by total weighted tasks (helpful for inspection)
# sector_sums = sector_sums.sort_values(by=[f'{handoff_var}_weighted'], ascending=False).reset_index(drop=True)
sector_sums = sector_sums.sort_values(by=['NAICS'], ascending=False).reset_index(drop=True)
display(sector_sums.head())

# 4) Merge the group sums back into master_df so each row has the sector totals
master_df = master_df.sort_values(by=['NAICS', 'occ_sectorEmpShare'], ascending=[False, False]).reset_index(drop=True)
# display(master_df.head())
master_df = master_df.merge(sector_sums, on=['NAICS', 'NAICS_TITLE'], how='left', suffixes=('', '_sectorTotal'))
master_df = master_df.sort_values(by=['NAICS', 'occ_sectorEmpShare'], ascending=[False, False]).reset_index(drop=True)
display(master_df.head())

,NAICS,NAICS_TITLE,task_semantic_dispersion_weighted,ai_fraction_weighted,human_E1_fraction_weighted,fragmentation_index_weighted,num_tasks_weighted
0,999300,"Local Government, excluding Schools and Hospit...",0.54,0.11,0.14,0.98,29.73
1,999200,"State Government, excluding Schools and Hospit...",0.53,0.13,0.19,0.97,33.27
2,999100,Federal Executive Branch (OEWS Designation),0.53,0.15,0.18,0.97,55.66
3,813900,"Business, Professional, Labor, Political, and ...",0.53,0.18,0.21,0.96,30.46
4,813400,Civic and Social Organizations,0.56,0.13,0.16,0.97,25.30


,Detailed_Occupation_Code,Detailed_Occupation_Title,task_semantic_dispersion,ai_fraction,human_E1_fraction,fragmentation_index,num_tasks,NAICS,NAICS_TITLE,sectorEmpShare,...,task_semantic_dispersion_weighted,ai_fraction_weighted,human_E1_fraction_weighted,fragmentation_index_weighted,num_tasks_weighted,task_semantic_dispersion_weighted_sectorTotal,ai_fraction_weighted_sectorTotal,human_E1_fraction_weighted_sectorTotal,fragmentation_index_weighted_sectorTotal,num_tasks_weighted_sectorTotal
0,33-3051,Police and Sheriff's Patrol Officers,0.57,0.12,0.05,0.95,41,999300,"Local Government, excluding Schools and Hospit...",0.03,...,0.06,0.01,0.00,0.10,4.10,0.54,0.11,0.14,0.98,29.73
1,33-2011,Firefighters,0.47,0.00,0.07,1.00,27,999300,"Local Government, excluding Schools and Hospit...",0.03,...,0.02,0.00,0.00,0.05,1.36,0.54,0.11,0.14,0.98,29.73
2,43-9061,"Office Clerks, General",0.52,0.38,0.24,0.86,21,999300,"Local Government, excluding Schools and Hospit...",0.03,...,0.01,0.01,0.01,0.02,0.58,0.54,0.11,0.14,0.98,29.73
3,49-9071,"Maintenance and Repair Workers, General",0.57,0.07,0.00,1.00,30,999300,"Local Government, excluding Schools and Hospit...",0.03,...,0.01,0.00,0.00,0.03,0.77,0.54,0.11,0.14,0.98,29.73
4,33-3012,Correctional Officers and Jailers,0.50,0.00,0.12,1.00,26,999300,"Local Government, excluding Schools and Hospit...",0.03,...,0.01,0.00,0.00,0.03,0.66,0.54,0.11,0.14,0.98,29.73


In [113]:
# Question 1: is higher hand-off measure associated with higher exposure?

# Regress ai_exposure_var on handoff_var
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Keep NAICS for clustering
reg_df = sector_sums[[
    'NAICS',
    f'{ai_exposure_var}_weighted',
    f'{handoff_var}_weighted',
]].dropna()

# Regression formula
formula = (
    f"{ai_exposure_var}_weighted ~ "
    f"{handoff_var}_weighted"
)
res = smf.ols(formula, data=reg_df).fit(
    cov_type='cluster',
    cov_kwds={'groups': reg_df['NAICS']}
)
print(res.summary())


                                OLS Regression Results                                
Dep. Variable:     human_E1_fraction_weighted   R-squared:                       0.111
Model:                                    OLS   Adj. R-squared:                  0.107
Method:                         Least Squares   F-statistic:                     27.79
Date:                        Mon, 10 Nov 2025   Prob (F-statistic):           2.96e-07
Time:                                13:20:46   Log-Likelihood:                 337.21
No. Observations:                         247   AIC:                            -670.4
Df Residuals:                             245   BIC:                            -663.4
Df Model:                                   1                                         
Covariance Type:                      cluster                                         
                                        coef    std err          z      P>|z|      [0.025      0.975]
----------------------------

In [ ]:
# Question 2: (Approach 1 -- without interactions) is higher hand-off measure associated with higher AI adoption, controlling for fragmentation and exposure?

# Regression: regress ai_outcome_var on handoff_var, ai_exposure_var, and fragmentation_index_var
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Keep NAICS for clustering
reg_df = sector_sums[[
    'NAICS',
    f'{ai_outcome_var}_weighted',
    f'{handoff_var}_weighted',
    f'{ai_exposure_var}_weighted',
    f'{fragmentation_index_var}_weighted'
]].dropna()

# Regression formula
formula = (
    f"{ai_outcome_var}_weighted ~ "
    f"{handoff_var}_weighted + {ai_exposure_var}_weighted + {fragmentation_index_var}_weighted"
)
res = smf.ols(formula, data=reg_df).fit(
    cov_type='cluster',
    cov_kwds={'groups': reg_df['NAICS']}
)
print(res.summary())

                             OLS Regression Results                             
Dep. Variable:     ai_fraction_weighted   R-squared:                       0.902
Model:                              OLS   Adj. R-squared:                  0.901
Method:                   Least Squares   F-statistic:                     556.6
Date:                  Mon, 10 Nov 2025   Prob (F-statistic):          2.68e-109
Time:                          13:20:47   Log-Likelihood:                 614.76
No. Observations:                   247   AIC:                            -1222.
Df Residuals:                       243   BIC:                            -1207.
Df Model:                             3                                         
Covariance Type:                cluster                                         
                                        coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------

In [ ]:
# Question 2: (Approach 2 -- with interactions) is higher hand-off measure associated with higher AI adoption, controlling for fragmentation and exposure?

# Regression: regress ai_outcome_var on handoff_var, ai_exposure_var, and fragmentation_index_var
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Keep NAICS for clustering
reg_df = sector_sums[[
    'NAICS',
    f'{ai_outcome_var}_weighted',
    f'{handoff_var}_weighted',
    f'{ai_exposure_var}_weighted',
    f'{fragmentation_index_var}_weighted'
]].dropna()

# Regression formula
formula = (
    f"{ai_outcome_var}_weighted ~ "
    f"{handoff_var}_weighted * {ai_exposure_var}_weighted * {fragmentation_index_var}_weighted"
)
res = smf.ols(formula, data=reg_df).fit(
    cov_type='cluster',
    cov_kwds={'groups': reg_df['NAICS']}
)
print(res.summary())

                             OLS Regression Results                             
Dep. Variable:     ai_fraction_weighted   R-squared:                       0.926
Model:                              OLS   Adj. R-squared:                  0.924
Method:                   Least Squares   F-statistic:                     416.3
Date:                  Mon, 10 Nov 2025   Prob (F-statistic):          1.77e-132
Time:                          13:20:47   Log-Likelihood:                 649.39
No. Observations:                   247   AIC:                            -1283.
Df Residuals:                       239   BIC:                            -1255.
Df Model:                             7                                         
Covariance Type:                cluster                                         
                                                                                                coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------